In [6]:
# Install necessary packages if not already installed
!pip install firecrawl
!pip install pymysql
!pip install python-dotenv
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Import necessary modules
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from firecrawl import FirecrawlApp

In [ ]:
# Load environment variables from .env (data base credentials)
load_dotenv()

In [ ]:
# Read the Firecrawl API key from the environment variables
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
if FIRECRAWL_API_KEY is None:
    raise Exception("FIRECRAWL_API_KEY not set in the .env file.")